In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../data/flights.parquet")

In [3]:
df.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2018-01-23,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,-5.0,...,1211.0,1249.0,7.0,1304,-8.0,0.0,-1.0,1300-1359,1,0.0
1,2018-01-24,Endeavor Air Inc.,ABY,ATL,False,False,1202,1157.0,0.0,-5.0,...,1210.0,1246.0,12.0,1304,-6.0,0.0,-1.0,1300-1359,1,0.0
2,2018-01-25,Endeavor Air Inc.,ABY,ATL,False,False,1202,1153.0,0.0,-9.0,...,1211.0,1251.0,11.0,1304,-2.0,0.0,-1.0,1300-1359,1,0.0
3,2018-01-26,Endeavor Air Inc.,ABY,ATL,False,False,1202,1150.0,0.0,-12.0,...,1207.0,1242.0,11.0,1304,-11.0,0.0,-1.0,1300-1359,1,0.0
4,2018-01-27,Endeavor Air Inc.,ABY,ATL,False,False,1400,1355.0,0.0,-5.0,...,1412.0,1448.0,11.0,1500,-1.0,0.0,-1.0,1500-1559,1,0.0


In [4]:
df.columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [5]:
target_cols = [col for col in df.columns.to_list() if "Del" in col]
y_columns = ["DepDelayMinutes"]
y = df[y_columns]
y.head()

,DepDelayMinutes
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [6]:
bool_cools = list(df.select_dtypes('bool').columns)
date_cools = list(df.select_dtypes('datetime').columns)
bool_cools.extend(date_cools)
bool_cools.extend(target_cols)
bool_cools

['Cancelled',
 'Diverted',
 'FlightDate',
 'DepDelayMinutes',
 'DepDelay',
 'ArrDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'ArrDelay',
 'ArrDel15',
 'ArrivalDelayGroups']

In [7]:
# Drop columns related  to delays, dates, bool columns due to the lack of information
df.drop(columns=bool_cools, inplace=True)
obj_cols = [col for col in df.columns if col not in df.select_dtypes('object')]
X = df[obj_cols]
X.shape, y.shape

((28339510, 33), (28339510, 1))

In [8]:
type(y.DepDelayMinutes)

pandas.core.series.Series

# Multivariate linear regression

In [9]:
from cuml.datasets.classification import make_classification
from cuml.model_selection import train_test_split
from cuml.ensemble import RandomForestRegressor as cuRF
from sklearn.metrics import accuracy_score
import cudf

In [10]:
X = cudf.from_pandas(X)
y = cudf.from_pandas(y.DepDelayMinutes)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/luis/miniconda3/envs/rapids-22.10/include/rmm/mr/device/cuda_memory_resource.hpp

In [ ]:
model = cuRF(max_depth=2, n_estimators=25)

## 10.3 hours running, not viable ...

In [ ]:
# scores = cross_val_score(forest, X_train, y_train, cv=5)

c:\Users\luigi\miniconda3\envs\pytorch\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\luigi\miniconda3\envs\pytorch\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


KeyboardInterrupt: 